In [ ]:
import mysql.connector
from IPython.display import display, Markdown

connection = mysql.connector.connect(
    user="root", password="secret", host="localhost", database="serlo"
)


def queryone(sql, parameters=[]):
    cursor = connection.cursor()
    cursor.execute(sql, parameters)
    return cursor.fetchone()


def queryall(sql, parameters=[]):
    cursor = connection.cursor()
    cursor.execute(sql, parameters)
    return cursor.fetchall()


def get_all_taxonomy_children(taxonomies, children=[]):
    for taxonomy in taxonomies:
        queried_children = queryall(
            """
           SELECT term_taxonomy.id FROM term_taxonomy
           JOIN uuid ON uuid.id = term_taxonomy.id
           JOIN uuid parent ON parent.id = parent_id
           WHERE parent_id = %s
               AND uuid.trashed = 0
               AND parent.trashed = 0
        """,
            [taxonomy],
        )
        queried_children = [child_row[0] for child_row in queried_children]
        children += list(set(queried_children))
        if len(queried_children) > 0:
            children = get_all_taxonomy_children(queried_children, children)
    return children


def get_all_entities_from_taxonomy(taxonomy):
    entities = queryall(
        """
        SELECT entity_id FROM term_taxonomy_entity
        JOIN uuid ON uuid.id = entity_id
        JOIN entity ON entity_id = entity.id
        JOIN entity_revision ON current_revision_id = entity_revision.id
        JOIN type ON entity.type_id = type.id
        JOIN license ON license.id = license_id
        WHERE term_taxonomy_id = %s
            AND trashed = 0
            AND type.name IN ("applet", "article", "course", "text-exercise",
                              "text-exercise-group", "video")
    """,
        [taxonomy],
    )
    entities = [entity_row[0] for entity_row in entities]
    return entities


def get_all_entities_from_taxonomy_tree(taxonomies):
    all_taxonomies = list(set(get_all_taxonomy_children(taxonomies)))
    all_entities = []
    for taxonomy in all_taxonomies:
        all_entities += get_all_entities_from_taxonomy(taxonomy)
    all_entities = list(set(all_entities))
    return all_entities


def get_entities_with_taxonomy(instance_id=None):
    return queryall(
        """
    SELECT entity.id FROM entity
    JOIN uuid ON uuid.id = entity.id
    JOIN type ON type_id = type.id
    JOIN entity_revision on current_revision_id = entity_revision.id
    JOIN license ON license.id = entity.license_id
    JOIN term_taxonomy_entity on term_taxonomy_entity.entity_id = entity.id
    WHERE trashed = 0
        AND (%s is NULL OR entity.instance_id = %s)
        AND type.name IN ("applet", "article", "course", "text-exercise",
                          "text-exercise-group", "video")
    GROUP BY entity.id
    """,
        [instance_id, instance_id],
    )